In [15]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from typing import List

In [16]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_fantracker"
api_name = "get_author_daily_fans"

In [17]:
fantracker_data = []
fantracker_columns = ["date_key", "fans_count", "influencer_id"]

In [18]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".json", "").split("_")[-1]),
} for blob in storage_client.list_blobs("0_raw_data",prefix="2_xingtu/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-11/get_author_daily_fans_0911_0.json, 1726037772347381>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-17/get_author_daily_fans_0917_0.json, 1726565275242714>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-18/get_author_daily_fans_0918_0.json, 1726633853919905>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-19/get_author_daily_fans_0919_0.json, 1726719806270115>,
  'date': datetime.datetime(2024, 9, 19, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-09/get_author_daily_fans_241009_0.json, 1728450610402889>,
  'date': datetime.datetime(2024, 10, 9, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_daily_fans_1011_0.json, 1728632611249040>,
  'date': datetime.datetime(20

In [19]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs("4_data_warehouse",prefix="1_xingtu/") if table_name == blob.name.split("/")[1] and len(blob.name.split("/")) == 4]
processed_blobs

[{'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_fantracker/2024-09-09/douyin_fantracker_240924_0.parquet, 1727172223527066>,
  'date': datetime.datetime(2024, 9, 9, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_fantracker/2024-09-10/douyin_fantracker_240918_0.parquet, 1726630470659978>,
  'date': datetime.datetime(2024, 9, 10, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_fantracker/2024-09-11/douyin_fantracker_240918_0.parquet, 1726630650706935>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_fantracker/2024-09-17/douyin_fantracker_240918_0.parquet, 1726631758393602>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_fantracker/2024-09-18/douyin_fantracker_240918_0.parquet, 1726653819862900>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': 0},
 {'blob': <

In [20]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 13):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch_number': 0,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_daily_fans_1011_0.json, 1728632611249040>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)}]
1


In [24]:
def get_author_daily_fans_data(raw_data):
    result = []
    for data in raw_data:
        profile_id = data["profile_id"]
        get_author_daily_fans = data.get("get_author_daily_fans")
        if get_author_daily_fans:
            daily = get_author_daily_fans.get("daily")
            if daily:
                for item in daily:
                    record = {}
                    record["date_key"] = item["date"]
                    record["fans_count"] = item["fans_cnt"]
                    record["influencer_id"] = profile_id
                    result.append(record)
    return result


In [25]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    fantracker_data = get_author_daily_fans_data(raw_data)
    if fantracker_data:
        df = pd.DataFrame(fantracker_data, columns=fantracker_columns)
        df.to_parquet("gs://4_data_warehouse/1_xingtu/douyin_fantracker/" + str(item["date"].date()) + "/douyin_fantracker_" + today.strftime("%y%m%d") + "_" + str(item["batch_number"]) + ".parquet")